In [1]:
!pip install -q diffusers transformers accelerate torch torchvision
!pip install -q opencv-python pillow


In [ ]:
# imports all required Python modules

import os
import torchiffusionPipeline, StableDiffusionImg2I
import cv2
import numpy as np
from PIL import Image
from diffusers import StableDmgPipeline
from IPython.display import Video


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [3]:
# ==============================
# Project Configuration
# ==============================

BASE_DIR = "/content/text_to_video_project"
FRAMES_DIR = os.path.join(BASE_DIR, "frames")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

IMAGE_SIZE = 512
NUM_FRAMES = 24        # ~6 seconds at 4 FPS
FPS = 4

BASE_SEED = 42
GUIDANCE_SCALE = 7.5

os.makedirs(FRAMES_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [4]:
prompt = input("Enter a text prompt: ").strip()

if len(prompt) < 3:
    raise ValueError("Prompt is too short to generate a video.")

print(f"Using prompt: {prompt}")


Enter a text prompt: cat play football
Using prompt: cat play football


In [5]:
MODEL_ID = "runwayml/stable-diffusion-v1-5"

# Text → Image (scene initialization)
txt2img = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16
).to("cuda")

# Image → Image (controlled evolution)
img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16
).to("cuda")

txt2img.enable_attention_slicing()
img2img.enable_attention_slicing()

print("Models loaded successfully.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Models loaded successfully.


In [6]:
# These modifiers work for ANY scene (nature, people, objects)
MOTION_MODIFIERS = [
    "same scene, cinematic still",
    "same scene, slight camera movement",
    "same scene, subtle depth shift",
    "same scene, gentle lighting change",
    "same scene, atmospheric motion"
]


In [7]:
generator = torch.Generator("cuda").manual_seed(BASE_SEED)

first_frame = txt2img(
    prompt=f"{prompt}, cinematic, high detail, stable composition",
    height=IMAGE_SIZE,
    width=IMAGE_SIZE,
    guidance_scale=GUIDANCE_SCALE,
    generator=generator
).images[0]

first_frame.save(os.path.join(FRAMES_DIR, "frame_000.png"))

previous_frame = first_frame


  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
for i in range(1, NUM_FRAMES):
    generator = torch.Generator("cuda").manual_seed(BASE_SEED + i)

    # Cycle through cinematic modifiers
    motion_prompt = (
        f"{prompt}, "
        f"{MOTION_MODIFIERS[i % len(MOTION_MODIFIERS)]}"
    )

    # Gradually increase freedom (prevents pixel jitter)
    strength = 0.25 + (0.35 * i / NUM_FRAMES)

    frame = img2img(
        prompt=motion_prompt,
        image=previous_frame,
        strength=strength,
        guidance_scale=GUIDANCE_SCALE,
        generator=generator
    ).images[0]

    frame_path = os.path.join(FRAMES_DIR, f"frame_{i:03d}.png")
    frame.save(frame_path)

    previous_frame = frame

    print(f"Generated frame {i} | strength={strength:.2f}")


  0%|          | 0/13 [00:00<?, ?it/s]

Generated frame 1 | strength=0.26


  0%|          | 0/13 [00:00<?, ?it/s]

Generated frame 2 | strength=0.28


  0%|          | 0/14 [00:00<?, ?it/s]

...

Generated frame 21 | strength=0.56


  0%|          | 0/28 [00:00<?, ?it/s]

Generated frame 22 | strength=0.57


  0%|          | 0/29 [00:00<?, ?it/s]

Generated frame 23 | strength=0.59


In [9]:
video_path = os.path.join(OUTPUT_DIR, "output_video.mp4")

frame_files = sorted(os.listdir(FRAMES_DIR))
first = cv2.imread(os.path.join(FRAMES_DIR, frame_files[0]))
h, w, _ = first.shape

writer = cv2.VideoWriter(
    video_path,
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (w, h)
)

for f in frame_files:
    img = cv2.imread(os.path.join(FRAMES_DIR, f))
    writer.write(img)

writer.release()

print("Video created:", video_path)


Video created: /content/text_to_video_project/output/output_video.mp4


In [10]:
Video(video_path, embed=True)
